In [1]:
%matplotlib widget
%config InlineBackend.figure_format = 'svg'
# %load_ext line_profiler # Example: %lprun -T lprof0 -f celeri.celeri.get_okada_displacements u_east, u_north, u_up = celeri.get_okada_displacements(segment.lon1.values[segment_idx], segment.lat1[segment_idx], segment.lon2[segment_idx], segment.lat2[segment_idx], segment.locking_depth[segment_idx], segment.burial_depth[segment_idx], segment.dip[segment_idx], command.material_lambda, command.material_mu, 1, 0, 0, station.lon, station.lat)

In [2]:
import addict
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from importlib import reload
from tqdm.notebook import tqdm

import celeri
celeri = reload(celeri)

# Plotting the global model is much much faster with tex fonts turned off
plt.rcParams['text.usetex'] = False

In [3]:
command_file_name = "./data/western_north_america/basic_command.json"
command, segment, block, meshes, station, mogi, sar = celeri.read_data(command_file_name)
station = celeri.process_station(station, command)
segment = celeri.process_segment(segment, command, meshes)
sar = celeri.process_sar(sar, command)
closure, block = celeri.assign_block_labels(segment, station, block, mogi, sar)
# celeri.plot_block_labels(segment, block, station, closure)

In [4]:
assembly = addict.Dict()
operators = addict.Dict()
assembly = celeri.merge_geodetic_data(assembly, station, sar) # Not sure this work correctly

# Build all linear operators
operators.block_rotation = celeri.get_block_rotation_operator(station)
operators.global_float_block_rotation = celeri.get_global_float_block_rotation_operator(station)
assembly, operators.block_motion_constraints = celeri.block_constraints(assembly, block, command)
assembly, operators.slip_rate_constraints = celeri.slip_rate_constraints(assembly, segment, block, command)
# operators.okada_segment_station = celeri.get_segment_station_operator_okada(segment, station, command)
# celeri.plot_segment_displacements(segment, station, command, segment_idx=0, strike_slip=1, dip_slip=0, tensile_slip=0, lon_min=235, lon_max=255, lat_min=30, lat_max=50, quiver_scale=1e-1)
operators.slip_rate_segment_block = celeri.get_fault_slip_rate_partials(segment, block)
operators.block_strain_rate, strain_rate_block_idx = celeri.get_strain_rate_centroid_operator(block, station, segment)
operators.mogi_station = celeri.get_mogi_operator(mogi, station, command)

# Triangles here
# operators.tri_station = celeri.get_tri_station_operator_okada(meshes, station, command)
# plt.figure()
# plt.imshow(np.log10(np.abs(operators.tri_station)), aspect="auto")
# plt.show()

# Get additional matrix shape information for assembly
assembly.index.sz_elastic = operators.okada_segment_station.shape # Not sure this is correct
assembly.index.sz_slip = operators.slip_rate_segment_block.shape # Not sure this is correct
assembly.index.sz_rotation = operators.block_rotation.shape # Not sure this is correct
assembly = celeri.station_row_keep(assembly) # Not sure this is correct

In [5]:
# Plot TDE displacements from a single element
# tri_idx = 0
# vel_east, vel_north, vel_up = celeri.get_tri_displacements(station.lon.to_numpy(), station.lat.to_numpy(), meshes, command.material_lambda, command.material_mu, tri_idx=0, strike_slip=1, dip_slip=0, tensile_slip=0)
# plt.figure()
# plt.plot(meshes[0].lon1[tri_idx], meshes[0].lat1[tri_idx], ".r")
# plt.plot(meshes[0].lon2[tri_idx], meshes[0].lat2[tri_idx], ".r")
# plt.plot(meshes[0].lon3[tri_idx], meshes[0].lat3[tri_idx], ".r")
# plt.quiver(station.lon, station.lat, vel_east, vel_north, scale=1e-3)
# plt.show()

In [6]:
# Get smoothing operator for a single mesh.
# Should we loop over all meshes and store them withing the meshes list?  Would 
share = celeri.get_shared_sides(meshes[0].verts)
tri_shared_sides_distances = celeri.get_tri_shared_sides_distances(share, meshes[0].x_centroid, meshes[0].x_centroid, meshes[0].x_centroid)
smoothing_matrix = celeri.get_tri_smoothing_matrix(share, tri_shared_sides_distances)
